## Load Libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as utils

from functools import partial as func_partial
from functools import reduce as func_reduce
from operator import mul as op_mul
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import PopulationBasedTraining
from os import cpu_count, path
from time import strftime

#This class contains DatasetClass and several helper functions
import ClassModule as cm

In [2]:
# Show number of avlaible CPU threads
# With mulithreading this number is twice the number of physical cores
cpu_av = cpu_count()
print("Number of available CPU's: {}".format(cpu_av))

Number of available CPU's: 12


In [3]:
# Set the number CPUS that should be used per trial and dataloader
# If set to 1 number of cucurrent training networking is equal to this number
# In case of training with GPU this will be limited to number of models training simultaneously on GPU
# So number of CPU threads for each trial can be increased 
cpus_per_trial = 1
gpus_per_trial = 0

In [4]:
def get_dataloader(train_ds, val_ds, bs):
    dl_train = utils.DataLoader(train_ds, batch_size=bs, shuffle=True, num_workers=cpus_per_trial-1)
    dl_val = utils.DataLoader(val_ds, batch_size=bs * 2, shuffle=True, num_workers=cpus_per_trial-1)
    return  dl_train, dl_val

## Instance Noise

In [5]:
# https://arxiv.org/abs/1610.04490
INSTANCE_NOISE = False

def add_instance_noise(data, std=0.01):
    return data + torch.distributions.Normal(0, std).sample(data.shape).to(device)

## Define the network
Get a network-candidate from the ASHA-scheduler first and use this notebook for hyperparameter tuning

In [6]:
class CNN(nn.Module):
    def __init__(self, input_dim=(2,20,20), num_in_features=5):
        super(CNN, self).__init__()
        self.feature_ext = nn.Sequential(
            nn.Conv2d(2,10, kernel_size=3, padding=2),
            nn.ReLU(),
            nn.Conv2d(10,10, kernel_size=3,  padding=2),
            nn.ReLU(),
            nn.Conv2d(10,10, kernel_size=3, padding=0),
            nn.ReLU(),
            nn.Conv2d(10,5, kernel_size=1, padding=0),
            nn.ReLU()
        )
        
        self.flatten = nn.Flatten()
        
        # Gives the number of features after the conv layer
        num_features_after_conv = func_reduce(op_mul, list(self.feature_ext(torch.rand(1, *input_dim)).shape))
        
        self.dense_nn = nn.Sequential(
            nn.Linear(num_features_after_conv + num_in_features, 4),
            nn.ReLU(),
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 32),
            nn.ReLU(),
            nn.Linear(32, 3),
            nn.ReLU()
        )
        
    def forward(self, cluster, clusNumXYEPt):
        cluster = self.feature_ext(cluster)
        x = self.flatten(cluster)
        x = torch.cat([x, clusNumXYEPt], dim=1)
        logits = self.dense_nn(x)
        return logits

## Implement train and validation loop
[0: 'ClusterN', 1:'Cluster', 2:'ClusterTiming', 3:'ClusterType', 4:'ClusterE', 5:'ClusterPt', 6:'ClusterModuleNumber', 7:'ClusterRow', 8:'ClusterCol', 9:'ClusterM02', 10:'ClusterM20', 11:'ClusterDistFromVert', 12:'PartE', 13:'PartPt', 14:'PartEta', 15:'PartPhi', 16:'PartIsPrimary', 17:'PartPID']

In [7]:
def train_loop(epoch, dataloader, model, loss_fn, optimizer, device="cpu"):

    size = len(dataloader.dataset)
    running_loss = 0.0
    epoch_steps = 0

    for batch, Data in enumerate(dataloader):
        Clusters = Data[0].to(device)
        Features = cm.unsqueeze_features(Data[1])
        Labels = Data[2]
        
        ClusterProperties = torch.cat([Features["ClusterE"], Features["ClusterPt"], Features["ClusterM02"]
                                      , Features["ClusterM20"], Features["ClusterDist"]], dim=1)
        ClusterProperties.to(device)
               
        # zero parameter gradients
        optimizer.zero_grad()
        
        # prediction and loss
        pred = model(Clusters, ClusterProperties)
        loss = loss_fn(pred, Labels["PartPID"].long())
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        epoch_steps += 1
        
        if batch % 10 == 9:
            print("[%d, %5d] loss: %.3f" % (epoch + 1, batch + 1,
                                            running_loss / epoch_steps))
            running_loss = 0.0        

In [8]:
def val_loop(epoch, dataloader, model, loss_fn, optimizer, device="cpu"):
    
    val_loss = 0.0
    val_steps = 0
    total = 0
    correct = 0
    size = len(dataloader.dataset)
    
    for batch, Data in enumerate(dataloader):
        with torch.no_grad():
            Clusters = Data[0].to(device)
            Features = cm.unsqueeze_features(Data[1])
            Labels = Data[2]
            ClusterProperties = torch.cat([Features["ClusterE"], Features["ClusterPt"], Features["ClusterM02"]
                                      , Features["ClusterM20"], Features["ClusterDist"]], dim=1)           
            ClusterProperties.to(device)
            
            pred = model(Clusters, ClusterProperties)
            correct += (pred.argmax(1) == Labels["PartPID"]).type(torch.float).sum().item()

            loss = loss_fn(pred, Labels["PartPID"].long())#.item()
            val_loss += loss.cpu().numpy()
            val_steps += 1
    
    with tune.checkpoint_dir(epoch) as checkpoint_dir:
        _path = path.join(checkpoint_dir, "checkpoint")
        torch.save((model.state_dict(), optimizer.state_dict()), _path)
        
    tune.report(loss=(val_loss / val_steps), accuracy= correct / size)

## Implement method for accuracy testing on test set

In [9]:
def test_accuracy(model, device="cpu"):
    
    dataset_test = cm.load_data_test()
    
    dataloader_test = utils.DataLoader(
        dataset_test, batch_size=4, shuffle=False, num_workers=2)
    
    correct = 0
    total = len(dataloader_test.dataset)
    
    with torch.no_grad():
        for batch, Data in enumerate(dataloader_test):
            Clusters = Data[0].to(device)
            Features = cm.unsqueeze_features(Data[1])
            Labels = Data[2]
            ClusterProperties = torch.cat([Features["ClusterE"], Features["ClusterPt"], Features["ClusterM02"]
                                      , Features["ClusterM20"], Features["ClusterDist"]], dim=1)            
            ClusterProperties.to(device)
            
            
            pred = model(Clusters, ClusterProperties)
            correct += (pred.argmax(1) == Labels["PartPID"]).type(torch.float).sum().item()

    return correct / total

## Implement training routine

In [10]:
def train_model(config, checkpoint_dir=None):
    
    # load model
    model = CNN()
    
    # check for avlaible resource and initialize device
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
    # send model to device
    model.to(device)
    
    # initialise loss function and opptimizer
    loss_fn = F.cross_entropy
    optimizer = torch.optim.Adam(model.parameters(),lr=config["lr"])
    
    # check whether checkpoint is available
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            path.join(checkpoint_dir, "checkpoint"))
        model.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)
        
    # load dataset
    dataset_train = cm.load_data_train()
    
    # split trainset in train and validation subsets
    test_abs = int(len(dataset_train) * 0.8)
    subset_train, subset_val = utils.random_split(
        dataset_train, [test_abs, len(dataset_train) - test_abs])

    # get dataloaders 
    dataloader_train, dataloader_val = get_dataloader(subset_train, subset_val, int(config["batch_size"]))
                                                      
    for epoch in range(100):
        train_loop(epoch, dataloader_train, model, loss_fn, optimizer, device=device)
        val_loop(epoch, dataloader_train, model, loss_fn, optimizer, device=device)                                              
    
    print("Finished Training")

## Setup all Ray Tune functionality and start training

In [11]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    
    # Setup hyperparameter-space to search
    config = {
        "lr": tune.loguniform(1e-4, 1e-1),
        "batch_size": tune.choice([2, 4, 8, 16, 32, 64])
    }

    # Init the scheduler
    scheduler = PopulationBasedTraining(
    time_attr="training_iteration",
    metric="accuracy",
    mode="max",
    perturbation_interval=10)
        
    
    # Init the Reporter
    reporter = CLIReporter(
        parameter_columns=["lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    
    #Get Current date and time
    timestr = strftime("%Y_%m_%d-%H:%M:%S")
    name = "ASHA-" + timestr
    
    # Init the run method
    result = tune.run(
        func_partial(train_model),
        name = name,
        resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        local_dir = "./Ray_Results",
        scheduler=scheduler,
        progress_reporter=reporter)
    
    # Find best trial and use it on the testset
    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(best_trial.last_result["accuracy"]))
    
    best_trained_model = CNN()
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)
    
    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)
    
    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))

In [12]:
main(num_samples=10, max_num_epochs=100, gpus_per_trial=gpus_per_trial)

/home/jhonerma/anaconda3/envs/cnn-env/lib/python3.9/site-packages/ray/_private/services.py:238: UserWarning: Not all Ray Dashboard dependencies were found. To use the dashboard please install Ray using `pip install ray[default]`. To disable this message, set RAY_DISABLE_IMPORT_WARNING env var to '1'.
  warnings.warn(warning_message)
2021-09-22 15:27:48,854	WARNING experiment.py:295 -- No name detected on trainable. Using DEFAULT.
2021-09-22 15:27:48,855	INFO registry.py:66 -- Detected unknown callable for trainable. Converting to class.


== Status ==
Memory usage on this node: 6.6/15.1 GiB
PopulationBasedTraining: 0 checkpoints, 0 perturbs
Resources requested: 1.0/12 CPUs, 0/0 GPUs, 0.0/5.87 GiB heap, 0.0/2.93 GiB objects
Result logdir: /home/jhonerma/ML-Notebooks/CNN/Ray_Results/ASHA-2021_09_22-15:27:47
Number of trials: 10/10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-------+-------------+--------------+
| Trial name          | status   | loc   |          lr |   batch_size |
|---------------------+----------+-------+-------------+--------------|
| DEFAULT_e138b_00000 | RUNNING  |       | 0.00211564  |           64 |
| DEFAULT_e138b_00001 | PENDING  |       | 0.000300854 |            2 |
| DEFAULT_e138b_00002 | PENDING  |       | 0.00691982  |            2 |
| DEFAULT_e138b_00003 | PENDING  |       | 0.0268359   |            8 |
| DEFAULT_e138b_00004 | PENDING  |       | 0.00757873  |           16 |
| DEFAULT_e138b_00005 | PENDING  |       | 0.0117591   |            8 |
| DEFAULT_e138b_00006 | PENDING  

(pid=25565) [2,    30] loss: 0.366
(pid=25565) [2,    40] loss: 0.275
(pid=25564) [2,    20] loss: 0.549
(pid=25564) [2,    30] loss: 0.366
(pid=25563) [3,    10] loss: 0.710
(pid=25565) [2,    50] loss: 0.220
(pid=25565) [2,    60] loss: 0.183
(pid=25559) [3,    10] loss: 1.099
(pid=25564) [2,    40] loss: 0.275
(pid=25564) [2,    50] loss: 0.220
(pid=25557) [4,    10] loss: 0.804
(pid=25555) [3,    10] loss: 1.099
(pid=25565) [2,    70] loss: 0.157
(pid=25560) [3,    10] loss: 1.068
(pid=25560) [3,    20] loss: 0.521
(pid=25562) [4,    10] loss: 1.099
(pid=25559) [3,    20] loss: 0.549
(pid=25559) [3,    30] loss: 0.366
(pid=25564) [2,    60] loss: 0.183
(pid=25564) [2,    70] loss: 0.157
(pid=25560) [3,    30] loss: 0.347
(pid=25563) [4,    10] loss: 0.779
(pid=25557) [5,    10] loss: 0.911
(pid=25555) [4,    10] loss: 1.099
(pid=25565) [3,    10] loss: 1.099
(pid=25565) [3,    20] loss: 0.549
(pid=25565) [3,    30] loss: 0.366
(pid=25562) [5,    10] loss: 1.099
(pid=25559) [4,    1

2021-09-22 15:27:53,054	INFO pbt.py:540 -- [exploit] transferring weights from trial DEFAULT_e138b_00009 (score 0.7210884353741497) -> DEFAULT_e138b_00008 (score 0.23129251700680273)
2021-09-22 15:27:53,055	INFO pbt.py:557 -- [explore] perturbed config from {} -> {}


(pid=25565) [5,    50] loss: 0.220
(pid=25565) [5,    60] loss: 0.183
(pid=25560) [7,    20] loss: 0.481
(pid=25559) [7,    30] loss: 0.366
(pid=25564) [5,    40] loss: 0.275
(pid=25563) [9,    10] loss: 0.712
(pid=25557) [11,    10] loss: 0.763
(pid=25565) [5,    70] loss: 0.157
(pid=25560) [7,    30] loss: 0.332
(pid=25562) [10,    10] loss: 1.099
(pid=25564) [5,    50] loss: 0.220
(pid=25555) [10,    10] loss: 1.099


...

(pid=25566) 2021-09-22 15:30:10,131	INFO trainable.py:382 -- Restored on 10.67.95.37 from checkpoint: /home/jhonerma/ML-Notebooks/CNN/Ray_Results/ASHA-2021_09_22-15:27:47/DEFAULT_e138b_00000_0_batch_size=64,lr=0.0021156_2021-09-22_15-27-49/checkpoint_tmp8c7aed/./
(pid=25566) 2021-09-22 15:30:10,131	INFO trainable.py:390 -- Current state after restoring: {'_iteration': 540, '_timesteps_total': None, '_time_total': 119.8959367275238, '_episodes_total': None}


(pid=25559) [99,    10] loss: 0.387
(pid=25559) [100,    10] loss: 0.446
(pid=25566) [1,    10] loss: 0.417
Result for DEFAULT_e138b_00006:
  accuracy: 0.891156462585034
  date: 2021-09-22_15-30-10
  done: true
  experiment_id: 770f51017fbf4dac9cb97cdddd05cb5f
  experiment_tag: 6_batch_size=4,lr=0.00065064@perturbed[]
  hostname: jhonerma-tuxedo
  iterations_since_restore: 100
  loss: 0.3570932825215095
  node_ip: 10.67.95.37
  pid: 25559
  should_checkpoint: true
  time_since_restore: 17.505584478378296
  time_this_iter_s: 0.16518926620483398
  time_total_s: 121.59690380096436
  timestamp: 1632317410
  timesteps_since_restore: 0
  training_iteration: 550
  trial_id: e138b_00006
  
(pid=25559) Finished Training
(pid=25566) [2,    10] loss: 0.335
(pid=25566) [3,    10] loss: 0.375
(pid=25566) [4,    10] loss: 0.364
(pid=25566) [5,    10] loss: 0.451
(pid=25566) [6,    10] loss: 0.351
== Status ==
Memory usage on this node: 6.4/15.1 GiB
PopulationBasedTraining: 129 checkpoints, 92 pertur

(pid=25566) [52,    10] loss: 0.651
(pid=25566) [53,    10] loss: 0.587
(pid=25566) [54,    10] loss: 0.566
(pid=25566) [55,    10] loss: 0.953
(pid=25566) [56,    10] loss: 1.053
(pid=25566) [57,    10] loss: 0.992
(pid=25566) [58,    10] loss: 0.974
(pid=25566) [59,    10] loss: 0.961
(pid=25566) [60,    10] loss: 0.968
(pid=25566) [61,    10] loss: 0.904
(pid=25566) [62,    10] loss: 0.889
(pid=25566) [63,    10] loss: 0.847
(pid=25566) [64,    10] loss: 0.865
(pid=25566) [65,    10] loss: 0.794
(pid=25566) [66,    10] loss: 0.818
(pid=25566) [67,    10] loss: 0.846
(pid=25566) [68,    10] loss: 0.854
(pid=25566) [69,    10] loss: 0.815
== Status ==
Memory usage on this node: 6.4/15.1 GiB
PopulationBasedTraining: 129 checkpoints, 92 perturbs
Resources requested: 1.0/12 CPUs, 0/0 GPUs, 0.0/5.87 GiB heap, 0.0/2.93 GiB objects
Result logdir: /home/jhonerma/ML-Notebooks/CNN/Ray_Results/ASHA-2021_09_22-15:27:47
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+---------------------+----

2021-09-22 15:30:26,597	INFO tune.py:561 -- Total run time: 157.75 seconds (157.53 seconds for the tuning loop).


== Status ==
Memory usage on this node: 6.4/15.1 GiB
PopulationBasedTraining: 129 checkpoints, 92 perturbs
Resources requested: 1.0/12 CPUs, 0/0 GPUs, 0.0/5.87 GiB heap, 0.0/2.93 GiB objects
Result logdir: /home/jhonerma/ML-Notebooks/CNN/Ray_Results/ASHA-2021_09_22-15:27:47
Number of trials: 10/10 (1 RUNNING, 9 TERMINATED)
+---------------------+------------+-------------------+------------+--------------+----------+------------+----------------------+
| Trial name          | status     | loc               |         lr |   batch_size |     loss |   accuracy |   training_iteration |
|---------------------+------------+-------------------+------------+--------------+----------+------------+----------------------|
| DEFAULT_e138b_00000 | RUNNING    | 10.67.95.37:25566 | 0.00757873 |           16 | 1.09861  |   0.210884 |                  640 |
| DEFAULT_e138b_00001 | TERMINATED |                   | 0.00757873 |           16 | 0.444517 |   0.891156 |                  130 |
| DEFAULT_e138b